# Libraries

In [1]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from keras_preprocessing.sequence import pad_sequences

import numpy as np
import os

# Preprocessing

In [2]:
# base path
directory_path = './labels'
# current directory
c_dir = os.getcwd()

# all actions
actions = np.array(sorted([folder for folder in os.listdir(directory_path) if os.path.isdir(os.path.join(directory_path, folder))])) # sorted to follow folder arrangement

# specific actions
# actions = np.array(['alligator', 'flower', 'kiss', 'listen', 'orange'])
# actions = np.array(['afternoon', 'house', 'again', 'open', 'kiss', 'sorry'])

In [3]:
# create a dictionary for int representation of actions
label_map = {label:num for num, label in enumerate(actions)}
label_map

{'afternoon': 0,
 'again': 1,
 'alligator': 2,
 'base': 3,
 'door': 4,
 'flower': 5,
 'hello': 6,
 'house': 7,
 'how': 8,
 'kiss': 9,
 'listen': 10,
 'open': 11,
 'orange': 12,
 'see': 13,
 'sorry': 14,
 'why': 15}

Note that at this point, we will not access the video folder, only the numpy folder.

In [4]:
sequences, labels = [], []  # sequence -> video, labels -> action
for action in actions:
    no_actions = len(os.listdir(os.path.join(c_dir, 'labels', action)))
    print('Opening path:', os.path.join(c_dir, 'labels', action))
    print(f'Number of instances: {no_actions}')
    for num in range(1, no_actions + 1):
        window = []         # window -> single frame
        file = str(action) + "_" + str(num)
        no_frames_per_action = len(os.listdir(os.path.join(c_dir, 'labels', action, file)))
        print(f'Number of frames in {file}: {no_frames_per_action}')
        for frame_num in range(1, no_frames_per_action + 1):
            res = np.load(os.path.join(c_dir, 'labels', action, file,  "{}.npy".format(frame_num)))     # res -> coordinate key points
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])
    print('-'*75)

Opening path: d:\GitHub\SSLrecognition\train_data\labels\afternoon
Number of instances: 40
Number of frames in afternoon_1: 31
Number of frames in afternoon_2: 30
Number of frames in afternoon_3: 30
Number of frames in afternoon_4: 30
Number of frames in afternoon_5: 31
Number of frames in afternoon_6: 31
Number of frames in afternoon_7: 31
Number of frames in afternoon_8: 30
Number of frames in afternoon_9: 31
Number of frames in afternoon_10: 31
Number of frames in afternoon_11: 31
Number of frames in afternoon_12: 31
Number of frames in afternoon_13: 31
Number of frames in afternoon_14: 31
Number of frames in afternoon_15: 31
Number of frames in afternoon_16: 31
Number of frames in afternoon_17: 31
Number of frames in afternoon_18: 31
Number of frames in afternoon_19: 31
Number of frames in afternoon_20: 31
Number of frames in afternoon_21: 31
Number of frames in afternoon_22: 31
Number of frames in afternoon_23: 31
Number of frames in afternoon_24: 31
Number of frames in afternoon_

In [5]:
# due to difference in number of frames, pad x and y
x = np.array(pad_sequences(sequences, dtype = 'float', padding = 'post', value = 0))
y = pad_sequences(to_categorical(labels).astype(int), dtype = 'int', padding = 'post', value = -1)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, stratify = y)

In [6]:
input_shape = (x_train.shape[1], x_train.shape[2])
x_train.shape

(392, 267, 225)

In [7]:
y_test

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

# Models

## LSTM

In [8]:
import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.callbacks import TensorBoard, TerminateOnNaN, EarlyStopping

from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [9]:
# for logging of data with TensorBoard
log_dir = os.path.join(c_dir, 'Logs')
tb_callback = TensorBoard(log_dir = log_dir)

# to end training when failure happens ie. loss == nan
term = TerminateOnNaN()

# to stop training early if there is no change in loss
early = EarlyStopping(monitor = 'loss', patience = 5)

In [10]:
def choose_lstm(n):
    if n == 1:
        model = Sequential()
        model.add(LSTM(512, return_sequences = True, input_shape = input_shape))
        model.add(Dropout(0.3))
        model.add(Bidirectional(LSTM(1024, return_sequences = True)))
        model.add(Dropout(0.2))
        model.add(Bidirectional(LSTM(512)))
        model.add(Dropout(0.2))
        model.add(Dense(actions.shape[0], activation = "softmax"))

        return model

    elif n == 2:
        model = Sequential()
        model.add(LSTM(64, return_sequences = True, input_shape = (117, 225)))
        model.add(LSTM(128, return_sequences = True))
        model.add(LSTM(64, return_sequences = False))
        model.add(Dense(64))
        model.add(Dense(32))
        model.add(Dense(8))
        model.add(Dense(actions.shape[0], activation = "softmax"))

        return model

    elif n == 3:
        model = Sequential()
        model.add(LSTM(128, return_sequences = True, input_shape = (117, 225)))
        model.add(Dropout(0.1))
        model.add(LSTM(64, return_sequences = False))
        model.add(Dropout(0.1))
        model.add(Dense(actions.shape[0], activation = "softmax"))

        return model
    
    elif n == 4:
        model = Sequential()
        model.add(LSTM(128, return_sequences=True, activation='relu', input_shape = input_shape))
        model.add(Dropout(0.2))
        model.add(LSTM(256, return_sequences=True, activation='relu'))
        model.add(Dropout(0.2))
        model.add(LSTM(256, return_sequences=False, activation='relu'))
        # model.add(BatchNormalization())
        model.add(Dense(256, activation='relu'))
        model.add(Dense(128, activation='relu'))
        model.add(Dense(64, activation='relu'))
        model.add(Dense(actions.shape[0], activation='softmax'))

        return model

In [11]:
model = choose_lstm(1)
opt = keras.optimizers.Adam(learning_rate = 0.01)
model.compile(optimizer = opt, loss = "categorical_crossentropy", metrics = ['categorical_accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 267, 512)          1511424   
                                                                 
 dropout (Dropout)           (None, 267, 512)          0         
                                                                 
 bidirectional (Bidirection  (None, 267, 2048)         12591104  
 al)                                                             
                                                                 
 dropout_1 (Dropout)         (None, 267, 2048)         0         
                                                                 
 bidirectional_1 (Bidirecti  (None, 1024)              10489856  
 onal)                                                           
                                                                 
 dropout_2 (Dropout)         (None, 1024)              0

In [12]:
model.fit(x_train, y_train, epochs = 100, batch_size = 32, callbacks = [term, early, tb_callback])

Epoch 1/100
13/13 [==============================] - 216s 17s/step - loss: 3.7511 - categorical_accuracy: 0.0791
Epoch 2/100
13/13 [==============================] - 269s 21s/step - loss: 2.7873 - categorical_accuracy: 0.1071
Epoch 3/100
13/13 [==============================] - 302s 23s/step - loss: 2.6033 - categorical_accuracy: 0.1301
Epoch 4/100
13/13 [==============================] - 315s 24s/step - loss: 2.6072 - categorical_accuracy: 0.1327
Epoch 5/100
13/13 [==============================] - 317s 24s/step - loss: 2.3209 - categorical_accuracy: 0.1862
Epoch 6/100
13/13 [==============================] - 323s 25s/step - loss: 2.3373 - categorical_accuracy: 0.1556
Epoch 7/100
13/13 [==============================] - 321s 25s/step - loss: 2.1845 - categorical_accuracy: 0.1684
Epoch 8/100
13/13 [==============================] - 324s 25s/step - loss: 2.0737 - categorical_accuracy: 0.2296
Epoch 9/100
13/13 [==============================] - 332s 25s/step - loss: 2.0916 - categorical_

In [13]:
# take model predictions
res = model.predict(x_test)

4/4 [==============================] - 13s 3s/step


In [14]:
res

array([[5.9500425e-03, 2.4015723e-04, 6.7491585e-04, ..., 3.1857928e-03,
        1.2047773e-02, 3.6042107e-03],
       [1.5126169e-04, 8.4223619e-05, 1.5702499e-01, ..., 3.7421294e-05,
        1.5718470e-04, 1.9804815e-04],
       [9.0988200e-05, 1.4010849e-04, 1.8820246e-01, ..., 3.1613090e-05,
        5.8050075e-05, 1.1823155e-04],
       ...,
       [1.2738142e-03, 2.0231931e-04, 1.3424119e-01, ..., 1.6723353e-04,
        1.0096573e-03, 9.4363757e-04],
       [4.0783216e-03, 9.6440507e-04, 1.4261860e-01, ..., 5.6744058e-04,
        1.2334653e-03, 1.9339509e-03],
       [7.3219635e-05, 1.8591584e-04, 1.3873626e-01, ..., 1.0038784e-06,
        2.0739371e-05, 6.7768683e-06]], dtype=float32)

In [15]:
y_true = np.argmax(y_test, axis = 1).tolist()
y_pred = np.argmax(res, axis = 1).tolist()

In [16]:
print(y_true)
print(y_pred)

[8, 10, 12, 12, 10, 5, 15, 15, 5, 10, 5, 5, 8, 14, 5, 9, 13, 4, 10, 9, 13, 8, 8, 14, 0, 9, 12, 2, 3, 0, 3, 2, 12, 3, 8, 12, 2, 0, 8, 13, 2, 10, 12, 10, 7, 9, 5, 3, 6, 5, 2, 2, 6, 10, 0, 5, 11, 12, 12, 6, 6, 0, 10, 13, 6, 10, 10, 8, 2, 14, 0, 15, 12, 5, 6, 9, 9, 0, 13, 9, 13, 13, 14, 5, 2, 6, 8, 6, 0, 9, 2, 2, 13, 1, 15, 9, 9, 12]
[8, 5, 5, 5, 5, 5, 13, 0, 5, 5, 12, 5, 8, 13, 5, 8, 13, 3, 5, 5, 13, 8, 8, 13, 0, 5, 12, 5, 3, 0, 3, 5, 5, 3, 8, 5, 5, 6, 8, 13, 5, 5, 5, 5, 0, 5, 5, 3, 0, 5, 5, 5, 13, 5, 8, 5, 3, 12, 5, 13, 13, 13, 12, 13, 13, 9, 5, 8, 5, 13, 3, 13, 5, 5, 13, 5, 5, 0, 13, 5, 13, 13, 13, 5, 5, 0, 8, 0, 0, 5, 5, 5, 13, 3, 13, 5, 12, 5]


In [17]:
print(f'Accuracy: {round(accuracy_score(y_true, y_pred)*100, 1)}%')

Accuracy: 35.7%


In [18]:
if accuracy_score(y_true, y_pred) >= 0.6:
    model.save('test_model.keras')
else:
    print('Model accuracy insufficient')

# keras.models.load_model("test_model.keras")

Model accuracy insufficient


In [19]:
# import os
# import shutil

# folder_path = './averaged_np_labels/'
# parent_files = os.listdir(os.path.join(folder_path))
# write_path = './labels/'

# for parent_file in parent_files:
#     parent_path = os.path.join(folder_path, parent_file)
#     export_path = os.path.join(write_path, parent_file)
#     os.mkdir(export_path)
#     print(f'Created new directory: {export_path}')
#     for i in range(1, len(os.listdir(os.path.join(folder_path, parent_file)))+1):
#         new_subfolder = f'{parent_file}_{i}'
#         os.mkdir(os.path.join(export_path, new_subfolder))
#         print(f'Created new subdirectory: {new_subfolder}')

#         source = os.path.join(parent_path, f'video{i}')
#         destination = os.path.join(os.path.join(export_path, new_subfolder))

#         sourcefolder = os.listdir(os.path.join(parent_path, f'video{i}'))
#         for file in sourcefolder:
#             file_to_copy = os.path.join(os.path.join(parent_path, f'video{i}'), file)
#             shutil.copy(file_to_copy, destination)